In [1]:
from ipywidgets import FileUpload, Output
import ipywidgets as widgets
from IPython.display import display, Javascript,  HTML
import sys
from io import StringIO
import pandas as pd
import plotly.express as px
import numpy as np 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.signal import find_peaks

out = Output()
upload = FileUpload(accept='.txt', multiple=True)

upload 

FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [2]:
surfaceInput = widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=10.0,
    step=0.1,
    description='Surface:',
    disabled=False
)
display(surfaceInput)

BoundedFloatText(value=0.1, description='Surface:', max=10.0, step=0.1)

In [3]:
# name = input('Please upload a file and a surface value. Press Enter when done.  \n')
def on_button_clicked(b):
    with out:
        out.clear_output()
        if(bool(upload.value)):  

            for uploaded_filename in upload.value:
                content = upload.value[uploaded_filename]['content']   
                print(uploaded_filename)

                s = "" 
                for line in content.split(b'\n'):
                    if "Source;Delais(s);Mesure ;Temps(s);Mesure/Source;Source/Mesure;Puissance" in line.decode('latin-1'):
                        s = ""
                    s +=line.decode('latin-1')

                df = pd.read_csv(StringIO(s), sep=";")
                df['J'] = df['Mesure ']*1000/surfaceInput.value
                df['Power'] = df['J']*df['Source']

                fig = make_subplots(rows=1, cols=2)

                fig1 = px.line(df,x="Source", y="J")
                fig2 = px.line(df,x="Source", y="Power")

                fig.add_trace(fig1['data'][0], row=1, col=1)
                fig.add_trace(fig2['data'][0], row=1, col=2)

                fig.add_trace(go.Scatter(x=[0],y=[np.interp(0, df.Source, df.J)],mode='markers'),row =1 , col =1)
                fig.add_trace(go.Scatter(x=[np.interp(0, df.J, df.Source)],y=[0],mode='markers'),row =1 , col =1)
                minPower = min(df.Power)
                fig.add_trace(go.Scatter(x=[np.array(df[df.Power == min(df.Power)].Source)[0]],y=[minPower],mode='markers'),row =1 , col =2) #df[df.Power == minPower]['Source']],y=[min(df.Power)],mode='markers'),row =1, col =2)


                # peaks, _ = find_peaks(df.Power, height=0)
                # fig.add_trace(go.Scatter(x=[df.Source[peaks[0]]],y=[df.Power[peaks[0]]],mode='markers'),row =1 , col =2)

                fig.update_layout(
                    showlegend=False,
                    autosize=False,
                    width=1500,
                    height=700,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=3
                    )
                )
                # Update xaxis properties
                fig.update_xaxes(title_text="Voltage [V]", row=1, col=1)
                fig.update_xaxes(title_text="Voltage [V]", row=1, col=2)

                # Update yaxis properties
                fig.update_yaxes(title_text="J [mA/cm^2] ", row=1, col=1)
                fig.update_yaxes(title_text="Power [mW/cm^2]", row=1, col=2)

                # Update title and height
                # fig.update_layout(title_text="Plots")
                fig.show()

                print("Jsc is: " + str(np.interp(0, df.Source, df.J)))
                print("Voc is: " + str(np.interp(0, df.J, df.Source)))
                print("Minimum power: " + str(min(df.Power)) + " at " + str(np.array(df[df.Power == min(df.Power)].Source)[0]) + " V and " + str(np.array(df[df.Power == min(df.Power)].J)[0]) + " mA/cm^2."  )
                # print("Maximum power: " + str(df.Power[peaks[0]]) + " at " + str(df.Source[peaks[0]]) + " V and " + str(df['Mesure '][peaks[0]]) + " A."  )

            upload.value.clear()
            upload._counter = 0

        else: 
            print("Empty file")

In [4]:
button = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', # (FontAwesome names without the `fa-` prefix)
)
button.on_click(on_button_clicked)
display(button)
out

Button(description='Plot', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()